In [15]:
import numpy as np 
import pandas as pd 
import os
import itertools

# plots
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# nltk
import nltk
from PIL import Image
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
from nltk.util import ngrams

# sklearn
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split

import re
from collections import Counter

# import tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



import requests
import json


In [16]:
# Covid tweets
covid=pd.read_csv('covid19_tweets.csv')

# 
sentiment=pd.read_csv('sentiment.csv',encoding = 'latin', header=None, names=['target','id','time','query','usr','text'])

# country code with GDP
country_code=pd.read_csv('2014_world_gdp_with_codes.csv') 

In [31]:
covid.head(10)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False
5,🎹 Franz Schubert,Новоро́ссия,🎼 #Новоро́ссия #Novorossiya #оставайсядома #S...,2018-03-19 16:29:52,1180,1071,1287,False,2020-07-25 12:27:06,#coronavirus #covid19 deaths continue to rise....,"['coronavirus', 'covid19']",Twitter Web App,False
6,hr bartender,"Gainesville, FL",Workplace tips and advice served up in a frien...,2008-08-12 18:19:49,79956,54810,3801,False,2020-07-25 12:27:03,How #COVID19 Will Change Work in General (and ...,"['COVID19', 'Recruiting']",Buffer,False
7,Derbyshire LPC,NaN,NaN,2012-02-03 18:08:10,608,355,95,False,2020-07-25 12:27:00,You now have to wear face coverings when out s...,NaN,TweetDeck,False
8,Prathamesh Bendre,NaN,"A poet, reiki practitioner and a student of law.",2015-04-25 08:15:41,25,29,18,False,2020-07-25 12:26:59,Praying for good health and recovery of @Chouh...,"['covid19', 'covidPositive']",Twitter for Android,False
9,Member of Christ 🇨🇳🇺🇸🇮🇳🇮🇩🇧🇷🇳🇬🇧🇩🇷🇺,👇🏻location at link below👇🏻,"Just as the body is one & has many members, & ...",2014-08-17 04:53:22,55201,34239,29802,False,2020-07-25 12:26:54,POPE AS GOD - Prophet Sadhu Sundar Selvaraj. W...,"['HurricaneHanna', 'COVID19']",Twitter for iPhone,False


In [19]:
covid.shape

(179108, 13)

In [18]:
country_code.head()

,COUNTRY,GDP (BILLIONS),CODE
0,Afghanistan,21.71,AFG
1,Albania,13.40,ALB
2,Algeria,227.80,DZA
3,American Samoa,0.75,ASM
4,Andorra,4.80,AND


In [20]:
country_code.shape

(222, 3)

In [40]:
# Below files are used for data cleaning - Retrieving countries with help of city codes/state codes...(optional)
state = json.loads('states.json')
# country = json.loads(requests.get("countries.json").text)
# city = json.loads(requests.get("cities.json").text)
# us_state_code = pd.read_csv('abbr-name.csv',names=['state_code','state'])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# All above mentioned data are functioned and retrieved in below file to get valid country name- without_country_name
without_country_name=pd.read_csv('../input/country-tweet/without_country_name.csv',low_memory=False)

In [30]:
print('Total tweets in this data: '+str(covid.shape[0]))
print('Total Unique Users in this data: '+str(covid['user_name'].nunique()) )

Total tweets in this data: 179108
Total Unique Users in this data: 92276


In [32]:
covid['country_name']=covid['user_location'].str.split(',').str[-1]
covid['only_date']=pd.to_datetime(covid['date']).dt.date

In [33]:
covid.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,country_name,only_date
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False,astroworld,2020-07-25
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False,NY,2020-07-25
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False,KY,2020-07-25
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False,Stuck in the Middle,2020-07-25
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False,Jammu and Kashmir,2020-07-25


In [34]:
#Keeping countries with valid country name

with_country_name = covid[covid['country_name'].isin(list(country_code['COUNTRY']))]
with_country_name['filtered_name']=covid['country_name']

In [35]:
with_country_name.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,country_name,only_date,filtered_name
11,Creativegms,"Dhaka,Bangladesh","I'm Motalib Mia, Logo -Logo Designer - Brandin...",2020-01-12 09:03:01,241,1694,8443,False,2020-07-25 12:26:50,Order here: https://t.co/4NUrGX6EmA\n\n#logo #...,"['logo', 'graphicdesigner', 'logodesign', 'log...",Twitter Web App,False,Bangladesh,2020-07-25,Bangladesh
30,Ms Paz,United States,NaN,2019-09-15 18:10:09,127,974,30217,False,2020-07-25 12:26:21,#FEMA acknowledges #PuertoRico lacks rebuilt h...,"['FEMA', 'PuertoRico', 'COVID19']",Twitter for iPhone,False,United States,2020-07-25,United States
35,Higher Health South Africa,South Africa,Higher Health is dedicated to promoting the he...,2014-05-23 06:18:35,1934,247,722,False,2020-07-25 12:26:09,Our CEO @RamneekHH live with @SASCO_Jikelele W...,NaN,Twitter for iPhone,False,South Africa,2020-07-25,South Africa
50,Gus McPop,Australia,Love Australia Fair and all things Australian,2014-06-06 22:26:36,580,551,100,False,2020-07-25 12:25:37,Coronavirus Testing Fiasco: St Mirren have ple...,NaN,TweetCaster for Android,False,Australia,2020-07-25,Australia
75,InvestmentGuruIndia,India,our ambition is to create universal platform 4...,2015-04-03 07:39:09,15635,35,2,False,2020-07-25 12:25:05,"Covid to shrink power sector growth, take disc...",['Industry'],Twitter Web App,False,India,2020-07-25,India


In [37]:
with_country_name.shape

(17525, 16)

In [ ]:
country=pd.DataFrame(country['countries'])
states =pd.DataFrame(state['states'])
city=pd.DataFrame(city['cities'])

all_world=pd.merge(city,states,left_on="state_id",right_on="id",how="left")
all_world=pd.merge(all_world,country,left_on="country_id",right_on="id",how="left")
all_world=pd.merge(all_world,us_state_code,left_on="name_y",right_on="state",how="left")

temp_df=all_world[['name_x','name']].rename(columns={'name_x':'place'})
temp_df=temp_df.append(all_world[['name_y','name']].rename(columns={'name_y':'place'}))
temp_df=temp_df.append(all_world[['sortname','name']].rename(columns={'sortname':'place'}))
temp_df=temp_df.append(all_world[['state_code','name']].rename(columns={'state_code':'place'}))
temp_df=temp_df.drop_duplicates()
temp_df.shape